In [1]:
%%sh
pygmentize sklearn-boston-housing.py

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import argparse, os

if __name__ == '__main__':
        
    parser = argparse.ArgumentParser()
    parser.add_argument('--normalize', type=bool, default=False)
    parser.add_argument('--test-size', type=float, default=0.1)
    parser.add_argument('--random-state', type=int, default=123)
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--training', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    
    args, _ = parser.parse_known_args()
    normalize = args.normalize
    test_size = args.test_size
    random_state = args.random_state
    model_dir  = args.model_dir
    training_dir = args.training

    filename = os.path.join(training_dir, 'housing.csv')
    data = pd.read_csv(filename)
    labels = data[['medv']]
    samples 

In [2]:
%%sh
pygmentize Dockerfile

FROM python:3.7

RUN pip3 install --no-cache scikit-learn numpy pandas joblib sagemaker-training


In [3]:
%%sh
aws ecr create-repository --repository-name sklearn-custom

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:613904931467:repository/sklearn-custom",
        "registryId": "613904931467",
        "repositoryName": "sklearn-custom",
        "repositoryUri": "613904931467.dkr.ecr.us-east-1.amazonaws.com/sklearn-custom",
        "createdAt": 1595252697.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [4]:
%%sh
export REGION=us-east-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:sklearn -f Dockerfile .
export IMAGE_ID=`docker images -q sklearn-custom:sklearn`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:latest
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn


Step 1/2 : FROM python:3.7
3.7: Pulling from library/python
e9afc4f90ab0: Pulling fs layer
989e6b19a265: Pulling fs layer
af14b6c2f878: Pulling fs layer
5573c4b30949: Pulling fs layer
11a88e764313: Pulling fs layer
ee776f0e36af: Pulling fs layer
0f6c875a5369: Pulling fs layer
a2726e1e2770: Pulling fs layer
029629755bc9: Pulling fs layer
0f6c875a5369: Waiting
ee776f0e36af: Waiting
029629755bc9: Waiting
a2726e1e2770: Waiting
11a88e764313: Waiting
5573c4b30949: Waiting
af14b6c2f878: Verifying Checksum
af14b6c2f878: Download complete
989e6b19a265: Verifying Checksum
989e6b19a265: Download complete
e9afc4f90ab0: Verifying Checksum
e9afc4f90ab0: Download complete
ee776f0e36af: Verifying Checksum
ee776f0e36af: Download complete
5573c4b30949: Verifying Checksum
5573c4b30949: Download complete
a2726e1e2770: Verifying Checksum
a2726e1e2770: Download complete
029629755bc9: Verifying Checksum
029629755bc9: Download complete
0f6c875a5369: Verifying Checksum
0f6c875a5369: Download complete
e9afc4f9

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [5]:
import sagemaker
from sagemaker.sklearn import SKLearn

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
sk = SKLearn(entry_point='sklearn-boston-housing.py',
             image_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:sklearn',
             role=role,
             train_instance_count=1, 
             train_instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training})

Creating tmpc56sg8wk_algo-1-2b7cx_1 ... 
Attaching to tmpc56sg8wk_algo-1-2b7cx_12mdone
algo-1-2b7cx_1  | 2020-07-20 13:46:14,004 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-2b7cx_1  | 2020-07-20 13:46:14,016 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-2b7cx_1  | 2020-07-20 13:46:14,027 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-2b7cx_1  | 2020-07-20 13:46:14,036 sagemaker-training-toolkit INFO     Invoking user script
algo-1-2b7cx_1  | 
algo-1-2b7cx_1  | Training Env:
algo-1-2b7cx_1  | 
algo-1-2b7cx_1  | {
algo-1-2b7cx_1  |     "additional_framework_parameters": {},
algo-1-2b7cx_1  |     "channel_input_dirs": {
algo-1-2b7cx_1  |         "training": "/opt/ml/input/data/training"
algo-1-2b7cx_1  |     },
algo-1-2b7cx_1  |     "current_host": "algo-1-2b7cx",
algo-1-2b7cx_1  |     "framework_module": null,
algo-1-2b7cx_1  |     "hosts": [
algo-1-2b7cx_1

In [6]:
%%sh
tar tvfz model.tar.gz

-rw-r--r-- ec2-user/ec2-user 784 2020-07-20 13:46 model.joblib
